# Coursera Scraper

Scraping the course details from Coursera

In [1]:
# Import libraries

import time
import re
from bs4 import BeautifulSoup as soup
import csv
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as expected
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from urllib.error import HTTPError

In [7]:
# Get the url for coursera with range of pages to be crawled
# Since Coursera can only show 1000 courses, we seperated the urls depends on levels.
urls_beginner = [
    'https://www.coursera.org/courses?query=&indices%5Bprod_all_products%5D%5BrefinementList%5D%5Blanguage%5D%5B0%5D=English&indices%5Bprod_all_products%5D%5BrefinementList%5D%5BproductDifficultyLevel%5D%5B0%5D=Beginner&indices%5Bprod_all_products%5D%5BrefinementList%5D%5BentityTypeDescription%5D%5B0%5D=Courses&indices%5Bprod_all_products%5D%5Bpage%5D='
    + str(page) + '&indices%5Bprod_all_products%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
    for page in range(1, 101, 1)
]
urls_mixed = [
    'https://www.coursera.org/courses?query=&indices%5Bprod_all_products%5D%5BrefinementList%5D%5Blanguage%5D%5B0%5D=English&indices%5Bprod_all_products%5D%5BrefinementList%5D%5BproductDifficultyLevel%5D%5B0%5D=Mixed&indices%5Bprod_all_products%5D%5BrefinementList%5D%5BentityTypeDescription%5D%5B0%5D=Courses&indices%5Bprod_all_products%5D%5Bpage%5D='
    + str(page) + '&indices%5Bprod_all_products%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
    for page in range(1, 92, 1)
]
urls_intermediate = [
    'https://www.coursera.org/courses?query=&indices%5Bprod_all_products%5D%5BrefinementList%5D%5Blanguage%5D%5B0%5D=English&indices%5Bprod_all_products%5D%5BrefinementList%5D%5BproductDifficultyLevel%5D%5B0%5D=Intermediate&indices%5Bprod_all_products%5D%5BrefinementList%5D%5BentityTypeDescription%5D%5B0%5D=Courses&indices%5Bprod_all_products%5D%5Bpage%5D='
    + str(page) + '&indices%5Bprod_all_products%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
    for page in range(1, 64, 1)
]
urls_advanced = [
    'https://www.coursera.org/courses?query=&indices%5Bprod_all_products%5D%5BrefinementList%5D%5Blanguage%5D%5B0%5D=English&indices%5Bprod_all_products%5D%5BrefinementList%5D%5BproductDifficultyLevel%5D%5B0%5D=Advanced&indices%5Bprod_all_products%5D%5BrefinementList%5D%5BentityTypeDescription%5D%5B0%5D=Courses&indices%5Bprod_all_products%5D%5Bpage%5D='
    + str(page) + '&indices%5Bprod_all_products%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
    for page in range(1, 12, 1)
]

In [37]:
def get_page_content(url, course=True):
    """
    url - base url to access desired course list page
    """
    try:
        profile = webdriver.FirefoxProfile()
        # 1 - Allow all images
        # 2 - Block all images
        # 3 - Block 3rd party images
        profile.set_preference("permissions.default.image", 2)
        driver = webdriver.Firefox(
            executable_path=r'/home/xinda/insight/notebooks/geckodriver', firefox_profile = profile)
        driver.get(url)
        if course == False:
            time.sleep(2)
        else:
            try:
                wait = WebDriverWait(driver, timeout=2)
                wait.until(expected.presence_of_all_elements_located(
                    (By.CLASS_NAME, 'occupation-name')))
                time.sleep(3)
            except:
                pass
        html = driver.page_source
        page_content = soup(html, 'html.parser')
        driver.close()  # Closed the browser opened in each loop.
        
        return page_content

    except HTTPError as e:
        print(e)

In [3]:
def get_course_type(course_tag):
    """
    Check course type if it is 'Course'. Only 'Course' is interested in.
    """
    if course_tag.startswith(r"/l"):
        course_type = 'Course'
    elif course_tag.startswith(r"/s"):
        course_type = 'specializations'
    else:
        course_type = 'professional-certificates'
    return course_type


def get_course_job(course_content):
    """
    Get the occupations who are interested in this course.
    """
    course_job = []
    course_job_temp = course_content.find_all(
        'li', {'class': 'occupation-name'})
    for link in course_job_temp:
        course_job.append(link.get_text())
    return course_job


def get_course_skill(course_content):
    """
    Get the skilled are expected to acquire in this course.
    """
    course_skill = []
    course_skill_temp = course_content.find_all(
        'span', {'class': 'Pill_56iw91 m-r-1s m-b-1s'})
    for link in course_skill_temp:
        course_skill.append(link.get_text())
    return course_skill


def get_course_description(course_content):
    course_description = []
    course_description_temp = course_content.find_all(
        'div', {'class': 'content-inner'})
    for link in course_description_temp:
        course_description.append(link.get_text())
    return course_description

In [4]:
def retry_get_job(course_job, course_link, times=3,):
    for x in range(0, times):
        if course_job == []:
            course_content = get_page_content(course_link)
            course_job = get_course_job(course_content)
        else:
            break
    return course_job

In [5]:
def scrap_coursera(csvfile='coursera.csv', urls='urls'):
    """
    Scrap all coursera courses.
    Save the result in csvfile.
    """
    with open(csvfile, 'w') as coursera:
        course_writer = csv.writer(coursera, delimiter=',')
        course_writer.writerow([
            'course_type', 'course_title', 'course_rating', 'course_link',
            'course_skill', 'course_job', 'course_description'
        ])  # Generate the header of csvfile.

        # loop through all the pages
        for url in urls:
            list_content = get_page_content(url, course=False)
            # Navigate to list of courses
            # Parse Data
            courses = list_content.find_all('li',
                                            {'class': 'ais-InfiniteHits-item'})
            for course in courses:
                try:
                    course_tag = course.a.get('href')
                    course_type = get_course_type(course_tag)
                    course_title = course.h2.get_text()
                    course_rating = course.find('span', {
                        'class': 'ratings-text'
                    }).get_text()
                    course_link = 'https://www.coursera.org%s' % course_tag
                    # Base course_link to get more course infromation
                    course_content = get_page_content(url=course_link)
                    course_skill = get_course_skill(course_content)
                    course_job = get_course_job(course_content)
                    course_job = retry_get_job(course_job, course_link)
                    print([
                        course_type, course_title, course_rating, course_link,
                        course_skill, course_job
                    ])
                    course_description = get_course_description(
                        course_content)
                    # Write into the file
                    course_writer.writerow([
                        course_type, course_title, course_rating, course_link,
                        course_skill, course_job, course_description
                    ])
                except AttributeError as e:
                    print(e)

In [6]:
scrap_coursera(csvfile='coursera_advanced.csv', urls=urls_advanced)
df_advanced = pd.read_csv('coursera_advanced.csv')
df_advanced

NameError: name 'urls_advanced' is not defined

In [52]:
scrap_coursera(csvfile='coursera_mixed.csv', urls=urls_mixed)
df_mixed = pd.read_csv('coursera_mixed.csv')
df_mixed

['Course', 'Machine Learning', '4.9', 'https://www.coursera.org/learn/machine-learning', ['Logistic Regression', 'Artificial Neural Network', 'Machine Learning (ML) Algorithms', 'Machine Learning'], ['Technical Leads', 'Machine Learning Engineers', 'Software Engineers', 'Risk Managers', 'Chief Technology Officers (CTOs)']]
['Course', 'The Science of Well-Being', '4.9', 'https://www.coursera.org/learn/the-science-of-well-being', ['Gratitude', 'Happiness', 'Meditation', 'Savoring'], ['Therapists', 'Real Estate Brokers', 'Paralegals', 'Nurses', 'Executive Assistants']]
['Course', 'Programming for Everybody (Getting Started with Python)', '4.8', 'https://www.coursera.org/learn/python', ['Python Syntax And Semantics', 'Basic Programming Language', 'Computer Programming', 'Python Programming'], ['Risk Managers', 'Data Analysts', 'Traders', 'Auditors', 'Financial Analysts']]
['Course', 'Python Data Structures', '4.9', 'https://www.coursera.org/learn/python-data', ['Python Syntax And Semantics

['Course', 'Introduction to Financial Accounting', '4.7', 'https://www.coursera.org/learn/wharton-accounting', ['Financial Accounting', 'Accounting', 'Financial Statement', 'Balance Sheet'], ['Accountants', 'Chief Financial Officers (CFOs)', 'Controllers', 'Directors of Operations', 'Financial Advisors']]
['Course', 'Storytelling and influencing: Communicate with impact', '4.9', 'https://www.coursera.org/learn/communicate-with-impact', [], ['Writers', 'Community Managers', 'Chief Financial Officers (CFOs)', 'Marketing Analysts', 'Editors']]
['Course', 'Introduction to Psychology', '4.9', 'https://www.coursera.org/learn/introduction-psych', ['Psychological Issues', 'Psychology', 'Social Psychology', 'Mental Illness'], ['Psychiatrists', 'User Experience Researchers', 'Customer Care Reps', 'Social Workers', 'HR Specialists', 'Psychiatrists', 'User Experience Researchers', 'Customer Care Reps', 'Social Workers', 'HR Specialists']]
['Course', 'Economics of Money and Banking', '4.9', 'https:

['Course', 'Understanding Clinical Research: Behind the Statistics', '4.8', 'https://www.coursera.org/learn/clinical-research', ['Clinical Research', 'Statistics', 'Statistical Analysis', 'Statistical Hypothesis Testing'], ['Medical Doctors', 'Pharmacists', 'Dentists', 'Scientists', 'Nurses']]
['Course', 'Version Control with Git', '4.7', 'https://www.coursera.org/learn/version-control-with-git', ['Version Control', 'Github', 'Bitbucket', 'Command-Line Interface'], ['Data Scientists', 'Software Engineers', 'Technical Leads', 'Biostatisticians', 'System Administrators', 'Data Scientists', 'Software Engineers', 'Technical Leads', 'Biostatisticians', 'System Administrators']]
['Course', 'Operations Analytics', '4.7', 'https://www.coursera.org/learn/wharton-operations-analytics', ['Simulation', 'Mathematical Optimization', 'Solver', 'Decision Tree'], ['Operations Analysts', 'Risk Managers', 'Business Analysts', 'Operations Managers', 'Data Analysts']]
['Course', 'Wind Energy', '4.7', 'http

['Course', 'The Global Financial Crisis', '4.8', 'https://www.coursera.org/learn/global-financial-crisis', ['Crisis', 'Global Financial Crisis', 'Financial Markets', 'Financial Crisis'], ['Economists', 'Traders', 'Financial Advisors', 'Controllers', 'Financial Analysts', 'Economists', 'Traders', 'Financial Advisors', 'Controllers', 'Financial Analysts']]
['Course', 'Data Visualization and Communication with Tableau', '4.7', 'https://www.coursera.org/learn/analytics-tableau', ['Data Analysis', 'Tableau Software', 'Data Visualization (DataViz)', 'Visualization (Computer Graphics)'], ['Data Analysts', 'Business Analysts', 'Marketing Analysts', 'Data Scientists', 'Financial Analysts']]
['Course', 'Understanding International Relations Theory', '4.7', 'https://www.coursera.org/learn/international-relations-theory', [], ['Translators', 'Paralegals', 'Assistant Directors', 'Journalists', 'Lawyers', 'Translators', 'Paralegals', 'Assistant Directors', 'Journalists', 'Lawyers']]
['Course', 'Intr

['Course', 'Quantitative Methods', '4.7', 'https://www.coursera.org/learn/quantitative-methods', [], ['User Experience Researchers', 'Researchers', 'Professors', 'Psychologists', 'Social Workers', 'User Experience Researchers', 'Researchers', 'Professors', 'Psychologists', 'Social Workers']]
['Course', 'Corporate Finance Essentials', '4.8', 'https://www.coursera.org/learn/corporate-finance-essentials', ['Corporate Finance', 'Financial Risk', 'Evaluation', 'Investment'], ['Chief Financial Officers (CFOs)', 'Controllers', 'Financial Analysts', 'Financial Advisors', 'Account Managers', 'Chief Financial Officers (CFOs)', 'Controllers', 'Financial Analysts', 'Financial Advisors', 'Account Managers']]
['Course', 'Introduction to Electronics', '4.7', 'https://www.coursera.org/learn/electronics', [], ['Technical Writers', 'Engineers', 'Machine Learning Engineers', 'Security Engineers', 'Technical Solutions Engineers']]
['Course', 'Software Processes and Agile Practices', '4.7', 'https://www.co

...

['Course', 'The Age of Sustainable Development', '4.8', 'https://www.coursera.org/learn/sustainable-development', ['Energy', 'Sustainable Energy', 'Urban Planning', 'Environmental Protection'], ['Chief Financial Officers (CFOs)', 'Economists', 'Process Analysts', 'Social Workers', 'Artists', 'Chief Financial Officers (CFOs)', 'Economists', 'Process Analysts', 'Social Workers', 'Artists']]
['Course', 'General Chemistry: Concept Development and Application', '4.6', 'https://www.coursera.org/learn/general-chemistry', [], ['Chemists', 'Scientists', 'Pharmacists', 'Biologists', 'Nurses', 'Chemists', 'Scientists', 'Pharmacists', 'Biologists', 'Nurses']]
['Course', 'Exploring Beethoven’s Piano Sonatas', '4.8', 'https://www.coursera.org/learn/beethoven-piano-sonatas', [], ['Musicians', 'Artists', 'Public Relations Specialists', 'Writers', 'Editors']]
['Course', 'Advanced Chemistry', '4.7', 'https://www.coursera.org/learn/advanced-chemistry', [], ['Chemists', 'Pharmacists', 'Scientists', 'Teach

['Course', 'How to Validate your Startup Idea', '4.4', 'https://www.coursera.org/learn/startup-idea', ['Lean Startup', 'Value Proposition', 'Investment', 'Entrepreneurship'], ['Managing Directors', 'Entrepreneurs', 'Technical Solutions Engineers', 'Chief Executive Officers (CEOs)', 'Marketing Specialists']]
['Course', 'Programming Mobile Applications for Android Handheld Systems: Part 2', '4.7', 'https://www.coursera.org/learn/android-programming-2', ['Interaction Technique', 'List Of Graphical User Interface Elements', 'User Interface Techniques', 'Android (Operating System) Software', 'Computer Programming'], ['Technical Support Associates', 'Software Engineers', 'Data Scientists', 'Data Analysts', 'Engineers']]
['Course', 'Voice Disorders: What Patients and Professionals Need to Know', '4.9', 'https://www.coursera.org/learn/voice-disorders', [], ['Therapists', 'Musicians', 'Medical Doctors', 'Teachers', 'Professors', 'Therapists', 'Musicians', 'Medical Doctors', 'Teachers', 'Profess

['Course', "Federalism & Decentralization: Evaluating Africa's Track Record", '4.6', 'https://www.coursera.org/learn/decentralization-africa', [], ['Lawyers', 'Filmmakers', 'Researchers', 'Project Managers', 'Consultants']]
['Course', 'Soul Beliefs: Causes and Consequences - Unit 2: Belief Systems', '4.6', 'https://www.coursera.org/learn/soulbeliefs2', [], ['Artists', 'Social Workers', 'Teachers', 'Accountants', 'Project Managers']]
['Course', 'Ubiquitous Learning and Instructional Technologies', '3.7', 'https://www.coursera.org/learn/ubiquitouslearning', [], ['Teachers', 'Professors', 'Project Managers']]
['Course', 'Approximation Algorithms Part II', '4.8', 'https://www.coursera.org/learn/approximation-algorithms-part-2', [], ['Machine Learning Engineers', 'Researchers', 'Data Scientists', 'Data Engineers', 'Software Engineers']]
['Course', 'Project: Writing a Research Paper', '4.8', 'https://www.coursera.org/learn/academic-writing-capstone', [], ['Professors', 'Researchers', 'Teache

['Course', "Hôtel “De l'étoile” - a hotel in crisis?", '4.5', 'https://www.coursera.org/learn/hotel-management-project', [], ['Account Managers']]
['Course', 'Survey Data Collection and Analytics Project (Capstone)', '3.5', 'https://www.coursera.org/learn/survey-data-capstone', [], ['Researchers']]
['Course', 'Preparing for the AP Physics 1 Exam', '4.3', 'https://www.coursera.org/learn/ap-physics-1', [], []]
['Course', 'Service Transformed: Lessons in U.S. Veteran Centered Care', '4.3', 'https://www.coursera.org/learn/va-health-care', [], []]
['Course', 'Conflict Management Project', '4.7', 'https://www.coursera.org/learn/conflict-management-capstone-project', [], []]
['Course', 'Capstone:  Create Value from Open Data', '3.6', 'https://www.coursera.org/learn/strategic-business-analytics-capstone', [], ['Data Analysts']]
['Course', 'Healthcare Marketplace Capstone', '4.8', 'https://www.coursera.org/learn/healthcare-marketplace-capstone', [], ['Medical Doctors']]
['Course', 'iMOOC102: Ma

,course_type,course_title,course_rating,course_link,course_skill,course_job,course_description
0,Course,Machine Learning,4.9,https://www.coursera.org/learn/machine-learning,"['Logistic Regression', 'Artificial Neural Net...","['Technical Leads', 'Machine Learning Engineer...","[""Machine learning is the science of getting c..."
1,Course,The Science of Well-Being,4.9,https://www.coursera.org/learn/the-science-of-...,"['Gratitude', 'Happiness', 'Meditation', 'Savo...","['Therapists', 'Real Estate Brokers', 'Paraleg...",['In this course you will engage in a series o...
2,Course,Programming for Everybody (Getting Started wit...,4.8,https://www.coursera.org/learn/python,"['Python Syntax And Semantics', 'Basic Program...","['Risk Managers', 'Data Analysts', 'Traders', ...",['This course aims to teach everyone the basic...
3,Course,Python Data Structures,4.9,https://www.coursera.org/learn/python-data,"['Python Syntax And Semantics', 'Data Structur...","['Traders', 'Risk Managers', 'Systems Engineer...",['This course will introduce the core data str...
4,Course,Indigenous Canada,4.8,https://www.coursera.org/learn/indigenous-canada,"['Art History', 'Art', 'History', 'Mythology']","['Librarians', 'Program Coordinators', 'Social...",['Indigenous Canada is a 12-lesson Massive Ope...
...,...,...,...,...,...,...,...
888,Course,Foundations of Teaching for Learning Capstone:...,4.3,https://www.coursera.org/learn/teaching-capstone,[],[],['This course is open to learners who have com...
889,Course,Marketing Analytics Capstone Project,3.7,https://www.coursera.org/learn/marketing-analy...,[],[],['This capstone project will give you an oppor...
890,Course,Cryptography II,4.7,https://www.coursera.org/learn/crypto2,[],"['Security Engineers', 'Machine Learning Engin...",['Learn about the inner workings of cryptograp...
891,Course,Drugs and the Brain,4.5,https://www.coursera.org/learn/drugs-and-brain,[],"['Paramedics', 'Psychiatrists', 'Psychologists...","['The neuroscience of drugs for therapy, for p..."


In [53]:
scrap_coursera(csvfile='coursera_intermediate.csv', urls=urls_intermediate)
df_intermediate = pd.read_csv('coursera_intermediate.csv')
df_intermediate

['Course', 'Neural Networks and Deep Learning', '4.9', 'https://www.coursera.org/learn/neural-networks-deep-learning', ['Artificial Neural Network', 'Backpropagation', 'Python Programming', 'Deep Learning'], ['Data Scientists', 'Machine Learning Engineers', 'Biostatisticians', 'Researchers', 'Data Engineers', 'Data Scientists', 'Machine Learning Engineers', 'Biostatisticians', 'Researchers', 'Data Engineers']]
['Course', 'Google Cloud Platform Fundamentals: Core Infrastructure', '4.7', 'https://www.coursera.org/learn/gcp-fundamentals', ['Google Compute Engine', 'Google App Engine (GAE)', 'Google Cloud Platform', 'Cloud Computing'], ['Technical Solutions Engineers', 'Security Engineers', 'Technical Leads', 'Data Engineers', 'Systems Engineers', 'Technical Solutions Engineers', 'Security Engineers', 'Technical Leads', 'Data Engineers', 'Systems Engineers']]
['Course', 'Introduction to TensorFlow for Artificial Intelligence, Machine Learning, and Deep Learning', '4.7', 'https://www.course

['Course', 'Networking and Security Architecture with VMware NSX', '4.8', 'https://www.coursera.org/learn/networking-security-architecture-vmware-nsx', [], ['Security Engineers', 'System Administrators', 'Technical Solutions Engineers', 'Systems Engineers', 'Technical Support Associates', 'Security Engineers', 'System Administrators', 'Technical Solutions Engineers', 'Systems Engineers', 'Technical Support Associates']]
['Course', 'Bayesian Statistics: From Concept to Data Analysis', '4.6', 'https://www.coursera.org/learn/bayesian-statistics', ['Statistics', 'Bayesian Statistics', 'Bayesian Inference', 'R Programming'], ['Biostatisticians', 'Data Scientists', 'Scientists', 'Researchers', 'Traders']]
['Course', 'Data Visualization with Python', '4.6', 'https://www.coursera.org/learn/python-for-data-visualization', ['Python Programming', 'Data Virtualization', 'Data Visualization (DataViz)', 'Matplotlib'], ['Data Analysts', 'Data Scientists', 'Business Analysts', 'Technical Solutions Eng

['Course', 'Finance for Non-Financial Managers', '4.5', 'https://www.coursera.org/learn/finance-for-non-financial-managers', ['Project', 'Financial Statement', 'Finance', 'Balance Sheet'], ['Directors of Operations', 'Chief Financial Officers (CFOs)', 'Operations Analysts', 'Managing Directors', 'Financial Analysts']]
['Course', 'Applied Data Science Capstone', '4.7', 'https://www.coursera.org/learn/applied-data-science-capstone', [], ['Data Analysts', 'Data Scientists', 'Auditors', 'Technical Leads', 'Systems Analysts', 'Data Analysts', 'Data Scientists', 'Auditors', 'Technical Leads', 'Systems Analysts']]
['Course', 'Fundamentals of Immunology: Innate Immunity and B-Cell Function', '4.6', 'https://www.coursera.org/learn/immunologyfundamentalsimmunitybcells', [], ['Biostatisticians', 'Scientists', 'Biologists', 'Paramedics', 'Pharmacists']]
['Course', 'Design-Led Strategy: Design thinking for business strategy and entrepreneurship', '4.7', 'https://www.coursera.org/learn/design-strate

['Course', 'Introduction to Structured Query Language (SQL)', '4.8', 'https://www.coursera.org/learn/intro-sql', ['Phpmyadmin', 'MySQL', 'Relational Database', 'SQL'], ['Process Analysts', 'Data Analysts', 'Risk Managers', 'Operations Analysts', 'Business Analysts']]
['Course', 'Getting Started With Application Development', '4.5', 'https://www.coursera.org/learn/getting-started-app-development', ['Cloud Storage', 'Data Store', 'Google Cloud Platform', 'Cloud Computing'], ['Technical Leads', 'Technical Solutions Engineers', 'Software Engineers', 'Chief Technology Officers (CTOs)', 'Consultants']]
['Course', 'Launching into Machine Learning', '4.6', 'https://www.coursera.org/learn/launching-machine-learning', ['Tensorflow', 'Bigquery', 'Machine Learning', 'Data Cleansing'], ['Data Scientists', 'Machine Learning Engineers', 'Biostatisticians', 'Chief Technology Officers (CTOs)', 'Data Engineers']]
['Course', 'Nutrition and Lifestyle in Pregnancy', '4.8', 'https://www.coursera.org/learn/n

['Course', 'Teaching EFL/ESL Reading: A Task Based Approach', '4.7', 'https://www.coursera.org/learn/esl-reading', [], ['Teachers', 'Translators', 'Editors', 'Customer Care Reps', 'Professors']]
['Course', 'Global Environmental Management', '4.6', 'https://www.coursera.org/learn/global-environmental-management', ['Urban Planning', 'Water Resource Development', 'Natural Resources', 'Green Technology'], ['Geologists', 'Retailers', 'Biologists', 'Researchers', 'Operations Analysts', 'Geologists', 'Retailers', 'Biologists', 'Researchers', 'Operations Analysts']]
['Course', 'Data Structures', '4.7', 'https://www.coursera.org/learn/data-structures', ['Binary Search Tree', 'Priority Queue', 'Hash Table', 'Stack (Abstract Data Type)', 'List'], ['Machine Learning Engineers', 'Software Engineers', 'Data Scientists', 'Data Engineers', 'Traders']]
['Course', 'Doing More with SAS Programming', '4.8', 'https://www.coursera.org/learn/sas-programming-advanced', [], ['User Experience Researchers', 'Bio

...

['Course', 'Software Design as an Abstraction', '4.8', 'https://www.coursera.org/learn/software-design-abstraction', [], ['Data Scientists', 'Software Engineers', 'Data Engineers', 'Project Managers', 'Teachers']]
['Course', 'Cryptographic Hash and Integrity Protection', '4.5', 'https://www.coursera.org/learn/cryptographic-hash-integrity-protection', [], ['Security Engineers', 'Machine Learning Engineers', 'Software Engineers', 'Data Analysts', 'Engineers']]
['Course', 'Building R Packages', '4.2', 'https://www.coursera.org/learn/r-packages', ['Programming Tool', 'Github', 'Continuous Integration', 'R Programming'], ['Biologists', 'Data Scientists', 'Economists', 'Data Analysts', 'Researchers', 'Biologists', 'Data Scientists', 'Economists', 'Data Analysts', 'Researchers']]
['Course', 'Strategic Innovation: Managing Innovation Initiatives', '4.8', 'https://www.coursera.org/learn/strategic-innovation-innovation-at-the-frontier', ['Management', 'Innovation', 'Disruptive Innovation', 'Inno

['Course', 'Analytical Solutions to Common Healthcare Problems', '4', 'https://www.coursera.org/learn/analytical-solutions-common-healthcare-problems', [], ['Data Analysts', 'Data Scientists', 'Medical Doctors', 'Product Managers', 'Business Analysts']]
['Course', 'Designing data-intensive applications', '3', 'https://www.coursera.org/learn/data-intensive-applications', [], ['Data Scientists', 'Machine Learning Engineers', 'Data Engineers', 'Software Engineers', 'Data Analysts']]
['Course', 'Visual Recognition & Understanding', '3.3', 'https://www.coursera.org/learn/visual-recognition', ['Deep Learning', 'Matlab', 'Machine Learning', 'Computer Programming', 'Computer Vision'], ['Machine Learning Engineers', 'Data Scientists', 'Data Engineers', 'Software Engineers', 'Researchers']]
['Course', 'Content Strategy for Professionals: Capstone Project', '4.6', 'https://www.coursera.org/learn/content-strategy-project', [], ['Marketing Specialists', 'Marketers']]
['Course', 'Packet Switching Ne

['Course', "Exploring Beethoven's Piano Sonatas Part 4", '5', 'https://www.coursera.org/learn/exploring-beethoven-piano-sonatas-4', [], ['Economists', 'Medical Doctors', 'Musicians', 'Teachers', 'Researchers']]
['Course', 'Presentation skills: Public Speaking Project', '4.6', 'https://www.coursera.org/learn/public-speaking-project', [], ['Accountants', 'Project Managers', 'Teachers', 'Data Scientists', 'Consultants']]
['Course', 'Water Supply and Sanitation Policy in Developing Countries Part 2: Developing Effective Interventions', '4.6', 'https://www.coursera.org/learn/water-part-2', [], ['Researchers', 'Professors', 'Project Managers', 'Chief Executive Officers (CEOs)', 'Engineers']]
['Course', 'Intellectual Change in Early China: Warring States and Han', '4.6', 'https://www.coursera.org/learn/intellectual-change-early-china-the-warring-states-han', [], ['Entrepreneurs', 'Marketers', 'Medical Doctors', 'Teachers', 'Professors', 'Entrepreneurs', 'Marketers', 'Medical Doctors', 'Teache

['Course', 'Mastering Software Development in R Capstone', '4.3', 'https://www.coursera.org/learn/r-capstone', [], ['Data Scientists']]
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
['Course', 'Capstone Value Creation through Innovation', '2', 'https://www.coursera.org/learn/capstone-value-creation-innovation', [], ['Software Engineers']]
'NoneType' object has no attribute 'get_text'
['Course', 'Capstone Project – Analyzing a Real Transaction', '5', 'https://www.coursera.org/learn/capstone-project-analyzing-a-real-transaction', [], ['Financial Analysts', 'Data Analysts', 'Project Managers']]
['Course', 'Leadership in Interprofessional Informatics', '4.5', 'https://www.coursera.org/learn/leadership-interprofessional-informatics', ['Leadership Dynamics', 'Professional Development'], ['Nurses']]
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
['Course', '

,course_type,course_title,course_rating,course_link,course_skill,course_job,course_description
0,Course,Neural Networks and Deep Learning,4.9,https://www.coursera.org/learn/neural-networks...,"['Artificial Neural Network', 'Backpropagation...","['Data Scientists', 'Machine Learning Engineer...","['If you want to break into cutting-edge AI, t..."
1,Course,Google Cloud Platform Fundamentals: Core Infra...,4.7,https://www.coursera.org/learn/gcp-fundamentals,"['Google Compute Engine', 'Google App Engine (...","['Technical Solutions Engineers', 'Security En...",['This course introduces you to important conc...
2,Course,Introduction to TensorFlow for Artificial Inte...,4.7,https://www.coursera.org/learn/introduction-te...,"['Computer Vision', 'Tensorflow', 'Machine Lea...","['Data Scientists', 'Machine Learning Engineer...",['If you are a software developer who wants to...
3,Course,Introduction to Data Science in Python,4.5,https://www.coursera.org/learn/python-data-ana...,"['Python Programming', 'Numpy', 'Pandas', 'Dat...","['Risk Managers', 'Data Scientists', 'Data Ana...",['This course will introduce the learner to th...
4,Course,Convolutional Neural Networks,4.9,https://www.coursera.org/learn/convolutional-n...,"['Facial Recognition System', 'Tensorflow', 'C...","['Data Scientists', 'Machine Learning Engineer...",['This course will teach you how to build conv...
...,...,...,...,...,...,...,...
592,Course,Capstone Project – Analyzing a Real Transaction,5.0,https://www.coursera.org/learn/capstone-projec...,[],"['Financial Analysts', 'Data Analysts', 'Proje...",['The Capstone Project is the final part of th...
593,Course,Leadership in Interprofessional Informatics,4.5,https://www.coursera.org/learn/leadership-inte...,"['Leadership Dynamics', 'Professional Developm...",['Nurses'],"['In this course, students in the health and i..."
594,Course,Career Total Fitness Annual Retreat,2.5,https://www.coursera.org/learn/career-fitness,[],[],"['In the business world, modern organizations ..."
595,Course,Advanced Speaking and Listening Project,5.0,https://www.coursera.org/learn/speaking-listen...,[],[],"[""Learners will present a well-organized acade..."


In [54]:
scrap_coursera(csvfile='coursera_biginner.csv', urls=urls_beginner)
df_biginner = pd.read_csv('coursera_biginner.csv')
df_biginner

['Course', 'AI For Everyone', '4.8', 'https://www.coursera.org/learn/ai-for-everyone', ['Workflow of Machine Learning projects', 'AI terminology', 'AI strategy', 'Workflow of Data Science projects'], ['Product Managers', 'Data Scientists', 'Machine Learning Engineers', 'Chief Technology Officers (CTOs)', 'Technical Solutions Engineers', 'Product Managers', 'Data Scientists', 'Machine Learning Engineers', 'Chief Technology Officers (CTOs)', 'Technical Solutions Engineers']]
['Course', 'SQL for Data Science', '4.6', 'https://www.coursera.org/learn/sql-for-data-science', ['Data Science', 'Data Analysis', 'Sqlite', 'SQL'], ['Data Analysts', 'Business Analysts', 'Risk Managers', 'Marketing Analysts', 'Data Scientists']]
['Course', 'Project Launch', '4.7', 'https://www.coursera.org/learn/project-management', ['Project Management', 'Scope (Project Management)', 'Communication', 'Work Breakdown Structure'], ['Program Coordinators', 'Project Managers', 'Product Managers', 'Consultants', 'Manage

['Course', 'Game Theory', '4.6', 'https://www.coursera.org/learn/game-theory-1', ['Game Theory', 'Backward Induction', 'Bayesian Game', 'Problem Solving'], ['Economists', 'Traders', 'Data Scientists', 'Researchers', 'Machine Learning Engineers']]
['Course', 'Data Science Methodology', '4.6', 'https://www.coursera.org/learn/data-science-methodology', [], ['Data Analysts', 'Data Scientists', 'Process Analysts', 'Business Analysts', 'Risk Managers']]
['Course', 'Learning to Teach Online', '4.7', 'https://www.coursera.org/learn/teach-online', ['Teaching', 'Online Learning', 'Evaluation', 'Planning'], ['Instructional Designers', 'Teachers', 'Professors', 'Technical Writers', 'Translators', 'Instructional Designers', 'Teachers', 'Professors', 'Technical Writers', 'Translators']]
['Course', 'Finance for Non-Finance Professionals', '4.8', 'https://www.coursera.org/learn/finance-for-non-finance', ['Financial Accounting', 'Capital Budgeting', 'Corporate Finance', 'Finance'], ['User Experience Re

['Course', 'Modern Art & Ideas', '4.7', 'https://www.coursera.org/learn/modern-art-ideas', ['Art History', 'Art', 'History', 'Art Direction'], ['Artists', 'Copywriters', 'Designers', 'Photographers', 'Graphic Designers']]
['Course', 'Business Transformation with Google Cloud', '4.7', 'https://www.coursera.org/learn/business-transformation-google-cloud', ['Organizational Culture', 'Business Transformation', 'Business Value', 'Business Strategy', 'Business Case Analysis'], ['Account Managers', 'Operations Managers', 'Technical Solutions Engineers', 'Consultants', 'Sales Associates']]
['Course', 'Understanding the Brain: The Neurobiology of Everyday Life', '4.9', 'https://www.coursera.org/learn/neurobiology', ['Brain', 'Neurological Disorders', 'Neurobiology', 'Neurology'], ['Psychiatrists', 'Therapists', 'Psychologists', 'Medical Doctors', 'Biologists', 'Psychiatrists', 'Therapists', 'Psychologists', 'Medical Doctors', 'Biologists']]
['Course', 'Organizational Analysis', '4.7', 'https://

['Course', 'AWS Fundamentals: Addressing Security Risk', '4.4', 'https://www.coursera.org/learn/aws-fundamentals-addressing-security-risk', [], ['Security Engineers', 'Risk Managers', 'IT Support', 'Operations Analysts', 'Chief Technology Officers (CTOs)']]
['Course', 'Introduction to Web Development', '4.6', 'https://www.coursera.org/learn/web-development', ['Css Code', 'Html', 'JavaScript', 'Cascading Style Sheets (CCS)'], ['Customer Care Reps', 'Technical Writers', 'Translators', 'Human Resources', 'Marketing Managers']]
['Course', 'Data Management for Clinical Research', '4.7', 'https://www.coursera.org/learn/clinical-data-management', ['Survey Design', 'Data Collection', 'Data Management', 'Clinical Data Management'], ['Pharmacists', 'Medical Doctors', 'Biostatisticians', 'Researchers', 'Dentists']]
['Course', "International Women's Health and Human Rights", '4.9', 'https://www.coursera.org/learn/womens-health-human-rights', ['Health Education', 'Reproductive Health', 'Community H

['Course', 'Supply Chain Logistics', '4.7', 'https://www.coursera.org/learn/supply-chain-logistics', ['Warehouse Management', 'Logistics Planning', 'Supply Chain', 'Inventory'], ['Operations Analysts', 'Operations Managers', 'Geologists', 'Process Analysts', 'Project Coordinators']]
['Course', 'Agile Meets Design Thinking', '4.7', 'https://www.coursera.org/learn/uva-darden-getting-started-agile', ['agile user stories', 'User Experience Design (UXD)', 'Agile Software Development', 'Product Management', 'Software Development'], ['User Experience Researchers', 'Product Managers', 'Instructional Designers', 'Project Managers', 'Designers', 'User Experience Researchers', 'Product Managers', 'Instructional Designers', 'Project Managers', 'Designers']]
['Course', 'Dino 101: Dinosaur Paleobiology', '4.9', 'https://www.coursera.org/learn/dino101', ['Biodiversity', 'Biology', 'Evolution', 'Paleontology'], ['Biologists', 'Retailers', 'Receptionists', 'Nurses', 'Editors']]
['Course', 'Business Wri

...

['Course', 'Developing the Opportunity for Corporate Entrepreneurs', '4.4', 'https://www.coursera.org/learn/corporate-entrepreneurs-opportunity', [], ['Entrepreneurs', 'Consultants', 'Data Analysts', 'Entrepreneurs', 'Consultants', 'Data Analysts']]
['Course', 'Mindfulness in Integrative Healthcare', '5', 'https://www.coursera.org/learn/mindfulness-integrative-healthcare', ['wellbeing', 'improved symptom management', 'whole-person care', 'Mindfulness', 'integrative medicine'], ['Therapists', 'Psychologists', 'Nurses', 'Medical Doctors', 'Entrepreneurs', 'Therapists', 'Psychologists', 'Nurses', 'Medical Doctors', 'Entrepreneurs']]
['Course', 'Sustainable Innovation for Subsistence Marketplaces', '4.4', 'https://www.coursera.org/learn/subsistence-marketplaces-1', [], ['Program Managers', 'Business Developers', 'Vice Presidents', 'Customer Care Reps', 'Technical Solutions Engineers']]
['Course', 'Forecasting, Budgeting, Territories, Evaluation and Legal/Ethical Issues', '2.3', 'https://ww

['Course', 'Effective Communication Capstone Project', '4.7', 'https://www.coursera.org/learn/effective-communication-capstone', [], ['Data Scientists']]
['Course', 'Visionary leadership, identity & motivation: Become a meaning maker', '4.8', 'https://www.coursera.org/learn/visionary-leadership-meaning-maker', ['Motivation', 'Communication', 'Leadership', 'Brand Management'], ['Therapists', 'HR Specialists', 'Founders', 'Product Managers', 'Entrepreneurs']]
['Course', 'Cyber Security in Manufacturing', '4.6', 'https://www.coursera.org/learn/cyber-security-manufacturing', [], ['Security Engineers', 'Data Analysts', 'Project Managers', 'Consultants', 'Chief Executive Officers (CEOs)']]
['Course', 'Data Analysis and Presentation Skills: the PwC Approach Final Project', '4.7', 'https://www.coursera.org/learn/data-analysis-project-pwc', [], ['Financial Analysts', 'Consultants', 'Data Analysts', 'Business Analysts', 'Project Managers', 'Financial Analysts', 'Consultants', 'Data Analysts', 'B

['Course', 'Stalin and Stalinism in Russian History', '4.7', 'https://www.coursera.org/learn/stalinism', [], ['Public Relations Specialists', 'Lawyers', 'Artists', 'Writers', 'Economists', 'Public Relations Specialists', 'Lawyers', 'Artists', 'Writers', 'Economists']]
['Course', 'America Through Foreign Eyes', '4.6', 'https://www.coursera.org/learn/america-through-foreign-eyes', [], ['Artists', 'Editors', 'Teachers', 'Lawyers', 'Data Scientists', 'Artists', 'Editors', 'Teachers', 'Lawyers', 'Data Scientists']]
['Course', 'Our Earth: Its Climate, History, and Processes', '4.6', 'https://www.coursera.org/learn/our-earth', [], ['Retailers', 'Marketing Specialists', 'Researchers', 'Teachers', 'Medical Doctors']]
['Course', 'Health Across the Gender Spectrum', '4.8', 'https://www.coursera.org/learn/health-gender-spectrum', ['Mental Health Education', 'Patient Care', 'Public Health', 'Medicine'], ['Nutritionists', 'Nurses', 'Psychologists', 'Medical Doctors', 'Therapists', 'Nutritionists', '

['Course', 'Innovation: From Creativity to Entrepreneurship Capstone', '4.3', 'https://www.coursera.org/learn/innovation-creativity-entrepreneurship-capstone', [], ['Consultants', 'Data Scientists', 'Project Managers', 'Managers', 'Engineers']]
['Course', 'Protecting Business Innovations via Strategy', '4.7', 'https://www.coursera.org/learn/protect-business-innovations-strategy', [], ['Entrepreneurs', 'Lawyers', 'Project Managers', 'Data Scientists', 'Consultants', 'Entrepreneurs', 'Lawyers', 'Project Managers', 'Data Scientists', 'Consultants']]
['Course', 'Career Planning:  A Pathway to Employment', '4.4', 'https://www.coursera.org/learn/career-planning', [], ['Marketers', 'Teachers', 'Consultants', 'Project Managers', 'Software Engineers']]
['Course', 'Revolutionary Ideas: Borders, Elections, Constitutions, Prisons', '5', 'https://www.coursera.org/learn/revolutionary-ideas-borders-elections-constitutions-prisons', [], ['Lawyers', 'Consultants', 'Teachers', 'Professors', 'Engineers']

,course_type,course_title,course_rating,course_link,course_skill,course_job,course_description
0,Course,AI For Everyone,4.8,https://www.coursera.org/learn/ai-for-everyone,"['Workflow of Machine Learning projects', 'AI ...","['Product Managers', 'Data Scientists', 'Machi...",['AI is not only for engineers. If you want yo...
1,Course,SQL for Data Science,4.6,https://www.coursera.org/learn/sql-for-data-sc...,"['Data Science', 'Data Analysis', 'Sqlite', 'S...","['Data Analysts', 'Business Analysts', 'Risk M...","[""As data collection has increased exponential..."
2,Course,Project Launch,4.7,https://www.coursera.org/learn/project-management,"['Project Management', 'Scope (Project Managem...","['Program Coordinators', 'Project Managers', '...",['Today’s fast track business and global envir...
3,Course,Digital Transformation,4.8,https://www.coursera.org/learn/bcg-uva-darden-...,"['Technology Disruption', 'Digital Trends', 'C...","['Chief Financial Officers (CFOs)', 'Process A...",['Digital transformation is a hot topic--but w...
4,Course,Technical Support Fundamentals,4.8,https://www.coursera.org/learn/technical-suppo...,"['Binary Code', 'Customer Support', 'Linux', '...","['Technical Support Associates', 'Communicatio...",['This course is the first of a series that ai...
...,...,...,...,...,...,...,...
969,Course,Doing Business in China Capstone,4.3,https://www.coursera.org/learn/doing-business-...,[],[],['Doing Business in China Capstone enables you...
970,Course,Fundamentals of Accounting Capstone,4.0,https://www.coursera.org/learn/accounting-fund...,[],[],['**You should complete all other courses in t...
971,Course,Contemporary India,4.1,https://www.coursera.org/learn/india,[],[],"[""This course presents some important vignette..."
972,Course,Journalism Skills for Engaged Citizens,4.5,https://www.coursera.org/learn/journalism-skills,[],[],"['This is a course in basic journalism skills,..."


In [10]:
import pandas as pd
df_biginner = pd.read_csv('coursera_biginner.csv')
df_biginner['course_job'].value_counts()

[]                                                                                                                                                                                                  25
['Teachers']                                                                                                                                                                                         5
['Teachers', 'Teachers']                                                                                                                                                                             3
['Teachers', 'Professors', 'Teachers', 'Professors']                                                                                                                                                 2
['Professors']                                                                                                                                                                                       2
     

In [ ]:
['Graphic Designers','Data Scientist','Security Engineer','Pharmacist','Marketing Specialist','Business Analyst','Accountant','Customer Care Rep','Biologist','Librarian','Entrepreneur','Teacher','Photographer']